In [1]:
import datetime as dt
import time
#import matplotlib
import matplotlib.pyplot as plt
import numpy as np
#import scipy as sp

import pyxu.abc as pxa
import pyxu.operator as pxop
import pyxu.opt.stop as pxos
#from pyxu.opt.solver.pgd import PGD

import pfw_hawkes as pyfwl
from hawkes_likelihood import HawkesLikelihood
import utils as ut

seed = 1  # None  # for reproducibility
if seed is None:
    seed = np.random.randint(0, 1000)
print(f"Seed: {seed}")

Seed: 1


In [2]:
%load_ext autoreload
%autoreload 2

%matplotlib qt

### Playing around with stuff

In [3]:
testarr = np.array([1, 9, 9, 9, 2, 9, 9, 9, 3, 9, 9, 9]) #M=3 => M*(M+1) = 12
#[kM + k, kM+k + M] pour la k'eme tranche
indices = [index for k in range(3) for index in range(k*3 + k + 1, k*3 + k + 3 + 1)]
print(indices)

print([testarr[(k*3 + k + 1):(k*3 + k + 3 + 1)] for k in range(0, 3)])
print(testarr[indices])

[1, 2, 3, 5, 6, 7, 9, 10, 11]
[array([9, 9, 9]), array([9, 9, 9]), array([9, 9, 9])]
[9 9 9 9 9 9 9 9 9]


In [4]:
# ne pas oublier que le vecteur de parametres theta est dans R^{M^2 + M}
testL1norm = ut.L1NormPartialReg(shape=(1,3*(3+1)), S=indices, regLambda=1)

In [5]:
print(testL1norm(testarr))
print(testL1norm.prox(testarr, tau=1))

81.0
[1. 8. 8. 8. 2. 8. 8. 8. 3. 8. 8. 8.]


In [6]:
supportIndices = [0, 4]
injection = pxop.SubSample(10, supportIndices).T

x = np.ones(len(supportIndices))

print('injection:', injection(x))

injection: [1. 0. 0. 0. 1. 0. 0. 0. 0. 0.]


In [7]:
testarr = np.array([1, 9, 9, 9, 2, 9, 9, 9, 3, 9, 9, 9]) #M=3 => M*(M+1) = 12
indices = np.array([k + k*3 for k in range(3)], dtype="int32")
print(indices)
print(testarr[indices])

[0 4 8]
[1 2 3]


In [54]:
dim1 = 2
dim2 = 3

l1normdim1 = pxop.L1Norm(dim=dim1)
l2normdim2 = pxop.L2Norm(dim=dim2)

x1 = np.ones(dim1)
x2 = 2*np.ones(dim2)
bigx = np.hstack([x1, x2])

#sumOp = pxop.Sum(arg_shape=)
normOp = pxop.hstack([l1normdim1, l2normdim2])
print(normOp(bigx))
print(l1normdim1(x1) + l2normdim2(x2))

[5.46410162]
[5.46410162]


In [ ]:
toyM = 4
ez = np.array([k + k*toyM for k in range(toyM)], dtype="int32")
print(ez)

ez2 = [index for k in range(toyM) for index in range(k*toyM + k + 1, k*toyM + k + toyM + 1)]
print(ez2)

### Todo
Compute the likelihood on all realizations:
- New likelihood = sum of all likelihoods
= E(A1 * theta) + ... + E(A50 * theta)
Rewrite it as Etilde(Atilde * theta) where 
Atilde = np.vstack([A1, ..., A50])
and Etilde = somme de E(i'eme tranche pour la i'eme realisation), i=1 -> 50

- understand why process 7 is zero
- understand why FW selects all atoms

### Generation of the source

In [96]:
path = "simulated_data/events1.csv"
beta = 1
hpL = HawkesLikelihood(path=path, beta=beta)
#hpL.plot_realization()

In [97]:
start = time.time()
lip = hpL.opA.estimate_lipschitz(method='svd')
print("Computation time of the Lipschitz constant: {:.4f}".format(time.time()-start))
print("Lipschitz constant:", lip)
#np.linalg.svd to compute the largest singular value which is = to the lipschitz constant for linear operators

Computation time of the Lipschitz constant: 0.0199
Lipschitz constant: 992.8829325886461


### Generation of the source (using multiple realizations)

In [7]:
totalProcesses = 5
beta = 1

paths = [f"simulated_data/events{i}.csv" for i in range(totalProcesses)]
allHPs = np.full(totalProcesses, None)

for i in range(totalProcesses):
    print(f"Initializing process {i+1}...")
    allHPs[i] = HawkesLikelihood(path=paths[i], beta=beta)
# j = 3
# print(allHPs[j].E(np.ones(allHPs[j].E.shape[1])))

Initializing process 1...
Initializing process 2...


KeyboardInterrupt: 

In [102]:
# Define big forward operator A: R^{M + M^2} -> R^{K1 + ... + Kn}
bigA = pxop.vstack([allHPs[i].opA for i in range(totalProcesses)])

# Define big E operator E1(x_1, ..., x_K1) + E2(x_{K1+1}, ..., x_{K1+K2}) + ...
# E: R^{K1 + ... + Kn} -> R
bigE = pxop.hstack([allHPs[i].E for i in range(totalProcesses)])

# Define big -log(likelihood operator)
bigNegLogL = bigE * bigA

In [104]:
# print(bigE)
# print(bigNegLogL)
# x = np.ones(bigE.shape[1])
# print(bigE(x))

In [29]:
start = time.time()
lipBig = bigA.estimate_lipschitz(method='svd')
print("Computation time of the Lipschitz constant: {:.4f}".format(time.time()-start))
print("Lipschitz constant:", lipBig)

Computation time of the Lipschitz constant: 0.0698
Lipschitz constant: 1838.2469414785728


### Defining solvers

In [114]:
# Parameter for reconstruction
#lambda_factor = 0.1

## Parameters of the solvers
# Base
remove = True
min_iterations = 1
# Stop
eps = 1e-4
tmax = 15.0
eps_dcv = 1e-2
# PFW
ms_threshold = 0.1  # original value 0.8, the lower the more atoms will HP select
init_correction = 1e-1
final_correction = 1e-6
correction_steps = 5

#lambda_ = lambda_factor * np.linalg.norm(op.T(measurements), np.infty)  # rule of thumb to define lambda
lambda_ = 10e1

In [115]:
pfw = pyfwl.PFWLasso(
    hpL,  # replace by likelihood operator hpL
    hpL.opA,  # replace by hpL.opA
    lambda_,
    ms_threshold=ms_threshold,
    init_correction_prec=init_correction,
    final_correction_prec=final_correction,
    remove_positions=remove,
    min_correction_steps=correction_steps,
    show_progress=False,
)

In [116]:
stop_crit = pxos.RelError(
    eps=eps,
    var="objective_func",
    f=None,
    norm=2,
    satisfy_all=True,
)
# alternative stopping criteria
dcv = pyfwl.dcvStoppingCrit(eps_dcv)

# Minimum number of iterations
min_iter = pxos.MaxIter(n=min_iterations)

# Maximum duration
max_duration = pxos.MaxDuration(t=dt.timedelta(seconds=tmax))

stop = (min_iter & stop_crit) | max_duration

# track DCV
track_dcv = pxos.AbsError(eps=1e-10, var="dcv", f=None, norm=2, satisfy_all=True)

### Solving

In [121]:
print("Polyatomic FW: Solving ...")
start = time.time()
pfw.fit(stop_crit= stop | track_dcv, diff_lipschitz=lip**2)
data_p, hist_p = pfw.stats()
time_p = time.time() - start
print("\tSolved in {:.3f} seconds".format(time_p))


Polyatomic FW: Solving ...
diff_lipschitz constant provided.
initial value: [-797.5526601]
new indices [ 1  3  5  6  8 12 14 15 17 19 20 21 23 24 26 30 32 33 35 37 38 39 41 42
 44 46 48 49 50 51 52 53 55 57 59 60 62 64 65 66 67 68 69 70 71]
actual new indices: 45
Dense iterate before rs_corr: [1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
Dense iterate after rs_corr: [1.00161829 0.00513695 0.         0.00572203 0.         0.00624103
 0.0050158  0.         0.0091334  1.00120595 0.         0.
 0.00310588 0.         0.0043483  0.00406902 0.         0.00672611
 1.00196186 0.00417666 0.00263908 0.00688028 0.         0.00803544
 0.00526807 0.         0.01147493 1.00135978 0.         0.
 0.00369953 0.         0.00575447 0.00481093 0.         0.00705664
 1.00243614 0.00966237 0.00331217 0.00784235 0.         0.01268018
 0.0

### Evaluation of the solve

In [122]:
print("Final value of dual certificate:\n\tPFW: {:.4f}".format(data_p["dcv"]))
print("Final value of objective function:\n\tPFW : {:.2f}".format(hist_p[-1][-1]))

Final value of dual certificate:
	PFW: 6.8982
Final value of objective function:
	PFW : -33730.02


In [123]:
def supp(arr):
    return np.where(np.abs(arr) > 1e-3)[0]

#print(data_p["x"][supp(data_p["x"])])
#print(data_p["x"])
for i in range(hpL.M):
    print(f"Process {i}:", data_p["x"][(i*(hpL.M + 1)):(i*(hpL.M + 1) + hpL.M + 1)])

# Arrange intensities in a M x M matrix
intensities = np.zeros((hpL.M, hpL.M))
for i in range(hpL.M):
    intensities[i, :] = data_p["x"][(i*(hpL.M + 1) + 1):(i*(hpL.M + 1) + hpL.M + 1)]

Process 0: [1.59395102 0.90311926 0.29753309 1.27304795 0.         1.83223911
 1.60411033 0.         2.94489869]
Process 1: [1.56539457 0.72069607 0.         1.12920051 0.         1.76318288
 1.5278339  0.         2.82445232]
Process 2: [1.61686166 0.82387509 0.34688742 1.36450867 0.44545306 1.98810619
 1.58696367 0.38958849 3.05270575]
Process 3: [1.57269691 0.72210645 0.         1.16285917 0.52619017 1.82296873
 1.59395561 0.40430423 2.8117425 ]
Process 4: [1.65350432 1.11502589 0.35880178 1.37449472 0.48413591 2.18559425
 1.67007861 0.38737573 3.07561127]
Process 5: [1.63580326 0.79960223 0.28593175 1.34650245 0.64442579 1.91903088
 1.93259361 0.51965471 3.09188686]
Process 6: [1.56912815 0.7725932  0.         1.14800797 0.         1.84946967
 1.57695571 0.39734656 2.73665621]
Process 7: [1.66211498 0.86562912 0.34199167 1.39138101 0.70429468 2.01938623
 2.11988675 0.51850486 3.60682632]


In [124]:
# Visualize solution
fig, ax = plt.subplots(1, 1, figsize=(5,5))
im = ax.imshow(intensities/(hpL.beta*hpL.beta), # la norme L1 est alpha/beta^2
                cmap='viridis', 
                interpolation='nearest', 
                extent=[1, hpL.M, hpL.M, 1])
fig.colorbar(im, ax=ax)  # Add a colorbar to show the scale
ax.set_title('L1 norms of the $h_{ij}$')
plt.show()